Reset id column

Standardize location_coordinates column

In [ ]:
UPDATE properties_2025_merged
SET location_coordinates = CASE
    -- Fix typos with extra "1" at the beginning
    WHEN location_coordinates LIKE '1 1917''%' THEN REPLACE(location_coordinates, '1 1917''', '1°17''')
    WHEN location_coordinates LIKE '1 1°17''%' THEN REPLACE(location_coordinates, '1 1°17''', '1°17''')
    
    -- Fix longitude typo (379° should be 37°)
    WHEN location_coordinates LIKE '%379°%' THEN REPLACE(location_coordinates, '379°', '37°')
    
    -- Fix "1921'42.1" typo (should be 1°21'42.1)
    WHEN location_coordinates LIKE '%1921''42.1%' THEN REPLACE(location_coordinates, '1921''42.1', '1°21''42.1')
    
    -- Empty to NULL
    WHEN location_coordinates = '' THEN NULL
    
    ELSE location_coordinates
END;

-- Replace 'and' with comma
UPDATE properties_2025_merged
SET location_coordinates = REGEXP_REPLACE(location_coordinates, '\s+and\s+', ', ', 'gi')
WHERE location_coordinates IS NOT NULL;

-- Replace '&' with comma
UPDATE properties_2025_merged
SET location_coordinates = REPLACE(location_coordinates, ' & ', ', ')
WHERE location_coordinates IS NOT NULL;

-- Normalize spacing
UPDATE properties_2025_merged
SET location_coordinates = REGEXP_REPLACE(location_coordinates, '\s+', ' ', 'g')
WHERE location_coordinates IS NOT NULL;

Client names standardization

In [ ]:
UPDATE properties_2025_merged
SET client_name = CASE
    -- Standardize KCB Bank variations
    WHEN LOWER(client_name) LIKE '%kcb bank%' THEN 'KCB Bank Kenya Limited'
    
    -- Standardize Co-operative Bank variations
    WHEN LOWER(client_name) LIKE '%co-operative bank%' 
         OR LOWER(client_name) LIKE '%co-op%bank%' THEN 'Co-operative Bank of Kenya Limited'
    
    -- Standardize Co-op Trust
    WHEN LOWER(client_name) LIKE '%co-op trust%' THEN 'Co-op Trust Investment Services Limited'
    
    -- Remove titles (Mr., Mrs., Dr., etc.) from individual names
    WHEN client_name LIKE 'Mr.%' THEN TRIM(REGEXP_REPLACE(client_name, '^Mr\.\s*', '', 'i'))
    WHEN client_name LIKE 'Mrs.%' THEN TRIM(REGEXP_REPLACE(client_name, '^Mrs\.\s*', '', 'i'))
    WHEN client_name LIKE 'Dr.%' THEN TRIM(REGEXP_REPLACE(client_name, '^Dr\.\s*', '', 'i'))
    WHEN client_name LIKE 'Ms.%' THEN TRIM(REGEXP_REPLACE(client_name, '^Ms\.\s*', '', 'i'))
    
    -- Keep others as is
    ELSE client_name
END;

Renumber index column

In [ ]:
ALTER TABLE valuations_2021_raw DROP COLUMN id;

ALTER TABLE valuations_2021_raw
ADD COLUMN id SERIAL PRIMARY KEY;